In [80]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from scipy.interpolate import CubicSpline

In [81]:
df = pd.read_excel('../data/raw/economics.xlsx')
print(df.shape)
df.tail()

(164, 17)


,Date,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b13,b14,b17,b18,B-23,B-24
159,1399-10-01,1849.0,338.2,2719.5,34761700.0,231454.8,637.7,38742.0,60343.0,42000.0,245364.0,112643.0,735000.0,232.0,233.0,361.566667,565095.273300
160,1400-01-01,1450.0,385.1,2281.2,37054000.0,179481.5,710.4,38695.0,63445.0,42000.0,234591.0,104427.0,715000.0,243.9,245.4,386.533333,597995.590905
161,1400-04-01,1655.0,433.5,2815.1,40676000.0,233059.8,792.7,46872.0,85115.0,42000.0,258319.0,113564.0,727100.0,276.4,276.7,422.433333,760275.629653
162,1400-07-01,1274.0,477.8,1901.2,44269900.0,163294.0,828.7,52183.0,74657.0,42000.0,276962.0,122811.0,738800.0,328.8,331.5,458.933333,729759.982808
163,1400-10-01,1524.0,519.1,2308.8,48324400.0,209124.2,846.8,68354.0,87897.0,42000.0,267002.0,122583.0,864300.0,344.1,352.4,480.266667,838688.748504


In [82]:
# # add new rows for ['1401-01-01', '1401-04-01', '1401-07-01', '1401-10-01', '1402-01-01', '1402-04-01'] in date column
# df.loc[len(df)] = ['1401-01-01'] + ([np.nan] * (df.shape[1] - 1))
# df.loc[len(df)] = ['1401-04-01'] + ([np.nan] * (df.shape[1] - 1))
# df.loc[len(df)] = ['1401-07-01'] + ([np.nan] * (df.shape[1] - 1))
# df.loc[len(df)] = ['1401-10-01'] + ([np.nan] * (df.shape[1] - 1))
# df.loc[len(df)] = ['1402-01-01'] + ([np.nan] * (df.shape[1] - 1))
# df.loc[len(df)] = ['1402-04-01'] + ([np.nan] * (df.shape[1] - 1))
# df.tail(10)

In [83]:
# Missing values
# drop columns with more than 10% missing values
cols_to_drop = []
for col in df.columns:
    print(f'{col}: {df[col].isna().sum() / len(df[col])}')
    if df[col].isna().sum() / len(df[col]) > 0.1:
        cols_to_drop.append(col)
print(cols_to_drop)

Date: 0.0
b1: 0.024390243902439025
b2: 0.21951219512195122
b3: 0.024390243902439025
b4: 0.0
b5: 0.07317073170731707
b6: 0.4146341463414634
b7: 0.12195121951219512
b8: 0.12195121951219512
b9: 0.0
b10: 0.012195121951219513
b13: 0.34146341463414637
b14: 0.0
b17: 0.024390243902439025
b18: 0.024390243902439025
B-23: 0.024390243902439025
B-24: 0.24390243902439024
['b2', 'b6', 'b7', 'b8', 'b13', 'B-24']


In [84]:
print(df.shape)
df.drop(columns=cols_to_drop, inplace=True)
df.shape

(164, 17)


(164, 11)

In [85]:
# interpolate missing values using cubic spline
for col in df.columns:
    if df[col].isna().sum() > 0:
        print("---" * 5)
        print(f"Before interpolation: {df[col].isna().sum()}")
        df[col] = df[col].interpolate(method='cubic')
        print(f"After interpolation: {df[col].isna().sum()}")

# bfill for remaining missing values
df.fillna(method='bfill', inplace=True)
df.isna().sum()

---------------
Before interpolation: 4
After interpolation: 0
---------------
Before interpolation: 4
After interpolation: 0
---------------
Before interpolation: 12
After interpolation: 0
---------------
Before interpolation: 2
After interpolation: 0
---------------
Before interpolation: 4
After interpolation: 4
---------------
Before interpolation: 4
After interpolation: 4
---------------
Before interpolation: 4
After interpolation: 4


Date    0
b1      0
b3      0
b4      0
b5      0
b9      0
b10     0
b14     0
b17     0
b18     0
B-23    0
dtype: int64

In [86]:
# print rows with missing values
df[df.isna().any(axis=1)]

,Date,b1,b3,b4,b5,b9,b10,b14,b17,b18,B-23


In [87]:
# plot each column based on the index
for col in df.columns:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df[col]))
    fig.update_layout(title=col)
    fig.show()

In [ ]:
# save the dataframe
df.to_csv('../data/processed/economics.csv', index=False)